In [1]:
import pandas as pd
from datetime import datetime
import pytz

In [2]:
fakealarm_df = pd.read_csv("01-tw-fakescada.csv")

# RDF transformation

In [3]:
#define key namespaces to be used in the mapping
famo = "http://ontology.eil.utoronto.ca/FAMO/famo/"
famo_app = "http://ontology.eil.utoronto.ca/FAMO/famo_hub_application/"
ex = "http://example.com/data/"
rdf = "http://www.w3.org/1999/02/22-rdf-syntax-ns#"
owl = "http://www.w3.org/2002/07/owl#"

#current datetime to be used for identifier suffix
dt_now = datetime.now().strftime("%d%b%Y_%H%M%S")

## Alarm mapping

* **TODO** test for null values before applying mappings

In [4]:
ttl_list = []
#prefixes
ttl_list.append(f"PREFIX famo: <{famo}>"+'\n')
ttl_list.append(f"PREFIX famo_app: <{famo_app}>"+'\n')
ttl_list.append(f"PREFIX ex: <{ex}>"+'\n')
ttl_list.append(f"PREFIX rdf: <{rdf}>"+'\n')
ttl_list.append(f"PREFIX owl: <{owl}>"+'\n')

for i in range(0,len(fakealarm_df)):
    #record instantiation and interpretation
    #scada record about a role
    ttl_list.append(f"ex:scada_rolerec_{fakealarm_df['ServingRole'][i]} rdf:type famo:Record."+'\n')
    #instantiated in scada
    ttl_list.append(f"ex:scada_rolerec_{fakealarm_df['ServingRole'][i]} famo:instantiatedIn ex:scada_datasystem."+'\n')
    ttl_list.append(f"ex:scada_rolerec_{fakealarm_df['ServingRole'][i]} famo:hasInterpretation ex:scada_roleinterpretation_{fakealarm_df['ServingRole'][i]}."+'\n')
    #denotes corresponding dhid (a role, with an entity number)
    ttl_list.append(f"ex:scada_roleinterpretation_{fakealarm_df['ServingRole'][i]} famo:denotes ex:dhid_{fakealarm_df['ServingRole'][i]}."+'\n')
    ttl_list.append(f"ex:dhid_{fakealarm_df['ServingRole'][i]} rdf:type famo:Role."+'\n')
    
    #record properties
    #represents role of asset in state (SCADA event)
    #alarm identifier based on alarm name and datetime epoch
    #datetime stamp to epoch and xsd:dateTimeStamp
    dt = datetime.strptime(fakealarm_df['Time'][i],'%Y-%m-%d %H:%M:%S')
    dt_epoch = (dt).strftime('%s')
    dt_xsd = dt.astimezone(pytz.timezone('America/Toronto')).isoformat()
    alarm_uuid = f"{fakealarm_df['EventName'][i].replace(' ','')}_{dt_epoch}"
    ttl_list.append(f"ex:scada_rolerec_{fakealarm_df['ServingRole'][i]} famo:representsRoleOfAssetInState ex:scada_{alarm_uuid}."+'\n')
    
    #may also insert a mapping that the alarm is of a more specific type, e.g.
    # ex:scada_{alarm_uuid} rdf:type famo:{fakealarm_df['EventName'][i].replace(' ','')}
    
    #subclass of scada alarm or event
    if (fakealarm_df['EventType'][i] == "Alarm"):
        ttl_list.append(f"ex:scada_{alarm_uuid} rdf:type famo:SCADAAlarmState."+'\n')
    elif (fakealarm_df['EventType'][i] == "Warning"):
        ttl_list.append(f"ex:scada_{alarm_uuid} rdf:type famo:SCADAWarningState."+'\n')
    
    #state time
    #currently mapping to time:hasTime (agnostic as to what kind of time relation (beginning/end/etc) this may be the most appropriate
    dt.isoformat()
    ttl_list.append(f"ex:scada_{alarm_uuid} famo_app:hasXSDDateTimeStamp '{dt_xsd}'."+'\n')

In [5]:
ttl_results = open("01_fakealarms_out.ttl", "a")
#for i in range(0,len(fakeasset_df)-1):
#record instantiation and interpretation
ttl_results.writelines(ttl_list)

ttl_results.close()